# Étude sur les facteurs qui déterminent la santé des enfants aux États-Unis

# Importation des libraries 

In [ ]:
import s3fs # pour récupérer les fichiers de l'espace de stockage S3
import pandas as pd # pour la gestion des dataframe
import geopandas as gpd # pour la gestion des données géographiques (fichiers .shp)
import matplotlib.pyplot as plt
import script.clean_data as cd
from sklearn.impute import KNNImputer
import numpy as np

## Récupération des bases de données

In [ ]:
# Maximum 15 secondes

# (depuis le stockage S3 d'un membre du groupe via le dossier public diffusion)
# Stocker les fichiers dans un dossier data/nsch

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

chemin_lecture_nsch = "inacampan/diffusion/Determinants_of_children-s_health/NSCH/"
chemin_ecriture_nsch = "data/nsch/"

# Lecture des fichiers des bases de données
dfs = cd.lecture_fichier_sas(fs, chemin_lecture_nsch, chemin_ecriture_nsch)

# récuperér le guide technique du questionnaire
# ce fichier inclut le nom de la variable, la question qu'elle encode, l'année du questionnaire
# mais également des notes pour la compréhension de la base de données initiale

guide = cd.lecture_fichier_csv(fs, f"{chemin_lecture_nsch}NSCH_Dictionary.csv", f"{chemin_ecriture_nsch}NSCH_Dictionary.csv")

# Element de précaution : GUIDE contient plusieurs index de meme nom
# mais les caractéristiques ne sont pas identiques

In [ ]:
# Maximum 2 secondes

# Récuperer les fichiers associés au shapefile - utiles pour les cartes (depuis le stockage S3 d'un membre du groupe)

chemin_lecture_map = "inacampan/diffusion/Determinants_of_children-s_health/Map/"
chemin_ecriture_map = "data/map/"

# lecture du fichier
gdf = cd.lecture_fichier_shapefile(fs, chemin_lecture_map, chemin_ecriture_map)

## Description des données et statistiques descriptives

Conformément au site qui héberge les bases de données NSCH, le questionnaire de présélection (NSCH-S1) déterminait si l’adresse représentait une résidence occupée et s’il y avait des enfants admissibles âgés de 0 à 17 ans vivant à cette adresse échantillonnée.

Le questionnaire thématique ("topical") comprenait des questions détaillées portant sur un enfant sélectionné au hasard dans le ménage. Les ménages recevaient l’un des trois questionnaires thématiques spécifiques à l’âge, en fonction de l’âge de l’enfant échantillonné : 
* NSCH-T1 (ou T1) pour les enfants âgés de 0 à 5 ans,
* NSCH-T2 (ou T2) pour les enfants âgés de 6 à 11 ans, ou
* NSCH-T3 (ou T3) pour les enfants âgés de 12 à 17 ans.

Le type du questionnaire est codé dans la variable ```FORMTYPE```.
Dans notre projet nous regardons uniquement le questionnaire "topical". 

In [ ]:
df = dfs["2023"] # regardons d'abord l'année 2023
df.describe()

In [ ]:
# La base de données de 2023 est dense, avec un total de 452 de variables sur 55162 individus (à travers les états-Unis).
# Sur l'ensemble des enfants dans l'échantillons : 
# (i) 21524 sont âgés de 0 à 5 ans,
# (ii) 18397 sont âgés de 6 à 11 ans et 
# (iii) 15241 sont âgés de 11 à 17 ans.

print(df["FORMTYPE"].value_counts())
print(df["FIPSST"].value_counts()[:5])

On regarde le pourcentage de valeurs manquantes par colonne, par ordre décroissant.
On observe des taux de valeurs manquantes très élevés pour certaines questions qui ne sont posées que dans des cas très spécifiques et qui dépendent d’une réponse précédente. (Cela est rassurant.)

Par exemple, pour la variable ```CYSFIB_SCREEN``` (avec un taux de 99.94% de valeurs manquantes), celle-ci encode la question :

**Was this condition identified through a blood test done shortly after birth? … These tests are sometimes called newborn screening.** Cette question est ignorée si ```CYSTFIB = 2```. La variables ```CYSFIB``` a un taux très très faible (0.19%) de valeurs manquantes et parmi les valeurs prises, ```CYSFIB!=2``` dans seulement 30 questionnaires.

Pour les variables ```DIABETES_DESC``` et ```DIABETES_CURR```, les questions qu'elles encodent sont ignorées dès lorsque ```DIABETES = 2``` (ce qui regroupe seulement 40 questionnaires).

In [ ]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100

print("Valeurs manquantes en pourcentage par colonne\n",
    missing_values)
print("Nombre de colonnes avec au moins 50% de valeurs manquantes\n",(missing_values >= 50).sum())

# Illustration de l'exemple de CYSTFIB et DIABETES
print("---------------------------------")
print("Informations sur la variable CYSTFIB_SCREEN : ", guide[guide["Variable"] == "CYSTFIB_SCREEN"]["Universe"].iloc[0], "\n",
    guide[guide["Variable"] == "CYSTFIB"]["Question"].iloc[0])
print("Valeurs manquantes pour la variable CYSTFIB : \n",
    (df["CYSTFIB"].isna().mean() * 100).round(2), "%")
print(df["CYSTFIB"].value_counts())

print("---------------------------------")
print("Informations sur la variable DIABETES_CURR : ", guide[guide["Variable"] == "DIABETES_CURR"]["Universe"].iloc[0], "\n",
    guide[guide["Variable"] == "DIABETES"]["Question"].iloc[0])
print("Valeurs manquantes pour la variable DIABETES : \n",
    (df["DIABETES"].isna().mean() * 100).round(2), "%")
print(df["DIABETES"].value_counts())

### Choix des variables d'intérêt

Dans le guide, les variables sont classées dans plusieurs grandes catégories, en fonction de la question de recherche et également de la base de données associée. En vue de notre base de données, on peut garder uniquement les variables de type "Topical" et "Operational" (colonne "Source").

In [ ]:
guide = guide[guide["Source"].isin({"Topical", "Operational"})]

years = {"2021", "2022", "2023", "2024"}

# On garde uniquement des variables qui sont communes aux quatre dataset NSCH
guide = guide[guide["Survey Years"].apply(
    lambda x: years <= set(y.strip() for y in str(x).split(","))
)]

# Cela réduit la base à 318 variables communes entre les databases
# Netoyer les dataframe NSCH avec cette contrainte, en rajoutant le code de l'état de résidance
# comme variable admissible.

variables_com = set(guide["Variable"])
variables_com = variables_com | {"FIPSST"}

for year in ["2023", "2022", "2021", "2024"]:
    df = dfs[year]
    df = df.loc[:, df.columns.isin(variables_com)]
    dfs[year] = df

In [ ]:
counts = guide.groupby(["Source", "Topic"])["Variable"].count().reset_index(name="count")
print("Nombre de variable par groupe de (Source, Topic)\n",counts.to_string())

In [ ]:
# Sélectionner des variables dans l'univers "All Children" => cad des questions posées à toutes les groupes d'age

filter_variables = {
    var for var in variables_com
    if guide[guide["Variable"] == var]["Universe"].item() == "All Children"
}

# Proposition : 
final_variables = {

    # topical variables
    "ACE1", "ACE10", "ACE11", "ARRANGEHC", "ATHOMEHC", "AVOIDCHG", "C4Q04", "CURRCOV",
    "BLINDNESS", "BLOOD", "BREATHING", "CAVITIES", "DOWNSYN", "FOODSIT", "HCABILITY", "HEART", "HOWMUCH",
    "K2Q30A", "K2Q31A", "K2Q60A", "K4Q23", "K8Q11", "K8Q34", "SCREENTIME", "VIDEOPHONE", "SWALLOWING",
    
    # operational variables
    "BIRTH_YR_F", "FIPSST", "FORMTYPE",
    "METRO_YN", "MPC_YN",
    "FWC", # child's weight
    "YEAR", "HHID"
}

cd.write_questions(final_variables, guide)

In [ ]:
for year in ["2024", "2023", "2022", "2021"]:
    df = dfs[year]
    df = df.loc[:, df.columns.isin(final_variables)]
    dfs[year] = df

In [ ]:
df = dfs["2024"]
df.describe()

In [ ]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100
print(missing_values)

In [ ]:
for var in df.columns:
    print("Variable : " + var + " prend " + str(len(df[var].unique())) + " valeurs uniques")
    print(df[var].unique())

# On a une variable qui est à moitié catégorielles - le ID de l'état
# On a une variable continue - la taille (FWC) : 0 valeurs manquantes

### Imputation

Pour s'y prendre, on utilise la librairie scikit-learn et une imputation via la méthode des K plus proches voisins : [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html). Nous utilisons un nombre de voisins égal à 3 (au lieu du 5 par défaut), par soucis de simplicité. De plus, nos variables manquantes étant catégorielles, un choix trop élevé de voisins à considérer n'est pas justifiable. 

In [ ]:
# Cette cellule prend environ 8 à 10 minutes pour s'executer
# On recommande d'executer directement la cellule suivante qui récupère ce résultat depuis le S3

years = ["2024", "2023", "2022", "2021"]
dfs_final = cd.impute_values_over_dataset(years, dfs)


In [ ]:
# Sauvegarde initiale sous S3 - via le compte inacampan - executé une seule fois 


MY_BUCKET = "inacampan"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/Determinants_of_children-s_health/NSCH/clean_data"

for name, df in dfs.items():
    path = f"{FILE_PATH_OUT_S3}/{name}.parquet"
    df.to_parquet(path, index=False) 
    print(f"{name} sauvegardé sur S3")


In [ ]:
dfs_final = {}

MY_BUCKET = "inacampan"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/Determinants_of_children-s_health/NSCH/clean_data"

# récuperer les dataframes stockés antérieurement dans le S3

for name in years:
    path = f"{FILE_PATH_OUT_S3}/{name}.parquet"
    dfs_final[name] = pd.read_parquet(path) 

In [ ]:
# tester que l'imputation a été bien effectuée 
# on trie en ordre décroissant le pourcentage de valeurs manquantes et on regarde la première valeur 

for year in years :
    df = dfs_final[year]
    missing_values = df.isna().mean().sort_values(ascending=False) * 100
    assert missing_values.iloc[0] == 0.0, f"Erreur : la première valeur est {missing_values.iloc[0]} au lieu de 0.0 pour {year}"